# Laboratorio 8.2

Integrantes
- Pedro Domínguez
- Eduardo Arróspide

## Crear la conexion a la BD
Se utilizaron los modulos: "sqlalchemy",  "psycopg2" y "ipython-sql"

In [2]:
import sqlalchemy as sa

# Create the connection
engine = sa.create_engine('postgresql://postgres:a@localhost:5432/dvdrentalcopy')

#Load the SQL extension
%load_ext sql
%config SqlMagic.displaycon = False

#Set up the connection:
%sql $engine.url

## P1

In [31]:
%%sql
DROP SCHEMA IF EXISTS lab82 CASCADE;

CREATE SCHEMA lab82;
SET search_path = lab82;

CREATE EXTENSION pg_trgm;

CREATE TABLE p1 (
    body text,
    body_indexed text
);

INSERT INTO p1
	SELECT
    	md5(random()::text)
	FROM (
		SELECT * FROM
			generate_series (1,100000) AS id
		) AS x;

UPDATE p1 SET body_indexed = body;

CREATE INDEX p1_search_idx ON p1 USING gin (body_indexed gin_trgm_ops);

Done.
Done.
Done.
Done.
Done.
100000 rows affected.
100000 rows affected.
Done.


[]

In [37]:
%%sql
SELECT COUNT(*) AS NO_INDEXED FROM p1 WHERE body LIKE '%abc%';

1 rows affected.


no_indexed
762


In [40]:
%%sql
SELECT COUNT(*) AS INDEXED FROM p1 WHERE body_indexed LIKE '%abc%';

1 rows affected.


indexed
762


## P2

In [43]:
%%sql

SET search_path = public;
DROP INDEX IF EXISTS idx_content_ts CASCADE;

ALTER TABLE film 
DROP COLUMN IF EXISTS content_ts;

ALTER TABLE film ADD COLUMN content_ts tsvector;

UPDATE film
SET content_ts = x.content_ts FROM (
	SELECT film_id,
			setweight(to_tsvector('english', title), 'A') || 
			setweight(to_tsvector('english', description), 'B') AS content_ts 
	FROM film
	) AS x
WHERE x.film_id = film.film_id;


CREATE INDEX idx_content_ts ON film USING gin(content_ts);

Done.
Done.
Done.
Done.
1000 rows affected.
Done.


[]

In [47]:
%%sql
SELECT title, description, ts_rank_cd(content_ts, query_ts) AS score 
FROM film, to_tsquery('english', 'man | woman') query_ts 
WHERE query_ts @@ content_ts
ORDER BY score DESC
LIMIT 10;

10 rows affected.


title,description,score
Beneath Rush,A Astounding Panorama of a Man And a Monkey who must Discover a Man in The First Manned Space Station,1.2
Platoon Instinct,A Thrilling Panorama of a Man And a Woman who must Reach a Woman in Australia,1.2
Stranger Strangers,A Awe-Inspiring Yarn of a Womanizer And a Explorer who must Fight a Woman in The First Manned Space Station,1.2
Glory Tracy,A Amazing Saga of a Woman And a Womanizer who must Discover a Cat in The First Manned Space Station,1.2
Garden Island,A Unbelieveable Character Study of a Womanizer And a Madman who must Reach a Man in The Outback,0.8
Dragonfly Strangers,A Boring Documentary of a Pioneer And a Man who must Vanquish a Man in Nigeria,0.8
Crow Grease,A Awe-Inspiring Documentary of a Woman And a Husband who must Sink a Database Administrator in The First Manned Space Station,0.8
Beverly Outlaw,A Fanciful Documentary of a Womanizer And a Boat who must Defeat a Madman in The First Manned Space Station,0.8
Bilko Anonymous,A Emotional Reflection of a Teacher And a Man who must Meet a Cat in The First Manned Space Station,0.8
Bikini Borrowers,A Astounding Drama of a Astronaut And a Cat who must Discover a Woman in The First Manned Space Station,0.8


In [48]:
%%sql
SELECT title, description, ts_rank_cd(fulltext, query_ts) AS score 
FROM film, to_tsquery('english', 'man | woman') query_ts 
WHERE query_ts @@ fulltext
ORDER BY score DESC
LIMIT 10;

10 rows affected.


title,description,score
Beneath Rush,A Astounding Panorama of a Man And a Monkey who must Discover a Man in The First Manned Space Station,0.3
Platoon Instinct,A Thrilling Panorama of a Man And a Woman who must Reach a Woman in Australia,0.3
Stranger Strangers,A Awe-Inspiring Yarn of a Womanizer And a Explorer who must Fight a Woman in The First Manned Space Station,0.3
Glory Tracy,A Amazing Saga of a Woman And a Womanizer who must Discover a Cat in The First Manned Space Station,0.3
Garden Island,A Unbelieveable Character Study of a Womanizer And a Madman who must Reach a Man in The Outback,0.2
Dragonfly Strangers,A Boring Documentary of a Pioneer And a Man who must Vanquish a Man in Nigeria,0.2
Crow Grease,A Awe-Inspiring Documentary of a Woman And a Husband who must Sink a Database Administrator in The First Manned Space Station,0.2
Beverly Outlaw,A Fanciful Documentary of a Womanizer And a Boat who must Defeat a Madman in The First Manned Space Station,0.2
Bilko Anonymous,A Emotional Reflection of a Teacher And a Man who must Meet a Cat in The First Manned Space Station,0.2
Bikini Borrowers,A Astounding Drama of a Astronaut And a Cat who must Discover a Woman in The First Manned Space Station,0.2


## P3

In [3]:
%%sql

SET search_path = public;
DROP TABLE IF EXISTS news CASCADE;
DROP INDEX IF EXISTS idx_content_ts CASCADE;

CREATE TABLE news(
    num integer,
    id integer,
    title text,
    publication text,
    author text,
    date text, 
    year float,
    month float,
    url text,
    content text
);


COPY news FROM 'C:\Francisco\OneDrive - UNIVERSIDAD DE INGENIERIA Y TECNOLOGIA\UTEC\UTEC CS\2022-I\Base-de-datos-II\Asignaciones\labs\8-indexes-postgres\data\articles1.csv' DELIMITER ',' CSV HEADER;

ALTER TABLE news ADD COLUMN content_ts tsvector;

UPDATE news
SET content_ts = x.content_ts FROM (
	SELECT id,
			setweight(to_tsvector('english', title), 'A') || 
			setweight(to_tsvector('english', content), 'B') AS content_ts 
	FROM news
	) AS x
WHERE x.id = news.id;

ALTER TABLE news ADD COLUMN content_ts_no_index tsvector;

UPDATE news
SET content_ts_no_index = x.content_ts FROM (
	SELECT id,
			setweight(to_tsvector('english', title), 'A') || 
			setweight(to_tsvector('english', content), 'B') AS content_ts 
	FROM news
	) AS x
WHERE x.id = news.id;

CREATE INDEX idx_content_ts ON news USING gin(content_ts);

Done.
Done.
Done.
Done.
50000 rows affected.
Done.
50000 rows affected.
Done.
50000 rows affected.
Done.


[]

In [ ]:
%%sql

SELECT title, content, ts_rank_cd(content_ts, query_ts) AS score 
FROM news, to_tsquery('english', 'trump | president') query_ts 
WHERE query_ts @@ content_ts
ORDER BY score DESC
limit 100;

In [5]:
%%sql

SELECT title, content, ts_rank_cd(content_ts_no_index, query_ts) AS score 
FROM news, to_tsquery('english', 'trump | president') query_ts 
WHERE query_ts @@ content_ts_no_index
ORDER BY score DESC
limit 100;

100 rows affected.


[('Trump Announces Plan That Does Little to Resolve His Conflicts of Interest', '  Donald Trump will not be taking necessary steps to resolve his conflicts of interest before he takes office. At his press conference today —  his f ... (59263 characters truncated) ... it, however, the situation is nevertheless fraught with conflicts between the three men’s personal and political interests. Back to table of contents', 114.200005),
 ('***Horse Race LiveWire*** Nevada Votes: Trump Beats Rubio, Cruz Combined - Breitbart', 'Welcome to Breitbart News’s daily live updates of the 2016 horse race. [Final Nevada results: Trump 46% Rubio 24% Cruz 21%. With Hispanic voters, Tru ... (119466 characters truncated) ... ng in the race complicates things for Rubio.  —   7:33 am  —   Nevada caucuses tonight. Trump has led in every poll, ranging from +13 to +26 points. ', 111.8),
 ('***Horse Race LiveWire*** GOP Candidates Debate in Michigan - Breitbart', 'Welcome to Breitbart News’s daily live updates of the 2016 horse race. [Donald Trump, Ted Cruz, Marco Rubio and John Kasich will participate in Fox N ... (60329 characters truncated) ... k is also typical of the Establishment inasmuch as it is a strategic blunder. Trump now has another Establishment hypocrite to use as a punching bag.', 106.0),
 ('Donald Trump’s Interests vs. America’s, Dubai Edition', 'Though the biggest controversy over the New Year’s Eve celebration at      Donald Trump’s Florida estate, was apparently whether or not Joe Scarborou ... (50820 characters truncated) ... it, however, the situation is nevertheless fraught with conflicts between the three men’s personal and political interests. Back to table of contents', 98.200005),
 ('***Horse Race LiveWire*** Trump, Clinton Predicted to Stomp Super Tuesday - Breitbart', 'Welcome to Breitbart News’s daily live updates of the 2016 horse race. Here’s a preview of what to expect on Super Tuesday. [ —    1:22: Math not on  ... (52652 characters truncated) ... MN. Alaska will be after midnight eastern,   8:30  —   Donald Trump and Hillary are predicted to all but sew up their respective nominations tonight.', 96.6),
 ('***Horse Race LiveWire*** GOP Candidates Debate in Houston - Breitbart', 'Welcome to Breitbart News’s daily live updates of the 2016 horse race. Donald Trump, Ted Cruz, Marco Rubio, John Kasich and Ben Carson debate in Hous ... (61652 characters truncated) ... , 2016,   —   7:27 am  —   Mark Halperin reports on “Morning Joe” that Donald Trump is not spending any money on any ads in any Super Tuesday states.', 95.200005),
 ('***Horse Race LiveWire*** GOP Debate Tonight! - Breitbart', 'Welcome to Breitbart News’s daily live updates of the 2016 horse race. [ —    Kasich tells CNN he doesn’t need a “breakout moment” in the debates. Wh ... (57742 characters truncated) ... very winning presidential candidate since 1964.   7:46  —   Tonight on CNN, Jake Tapper moderates what will hopefully be the final Republican debate.', 94.4),
 ('***Horse Race LiveWire*** Chris Christie Endorses Donald Trump - Breitbart', 'Welcome to Breitbart News’s daily live updates of the 2016 horse race.[ —    11. 20: Even if Donald Trump wins the GOP nomination, the general electi ... (35009 characters truncated) ... on artist” at least 5 times during our @CBSThisMorning interview pic. twitter.   —   CBS This Morning (@CBSThisMorning) February 26, 2016,  —    —   ', 85.4),
 ('***Horse Race LiveWire*** 2 Days to South Carolina: Pope Francis Shifts the Race’s Focus - Breitbart', 'Welcome to Breitbart News’s daily live updates of the 2016 horse race. [ —    11:00 Trump speaks glowingly about his five children. He says he was a  ... (149058 characters truncated) ... nd national outlets, Garner said Sanders, who also endorsed Jesse Jackson’s 1988 presidential campaign, supported blacks before it was cool to do so.', 84.0),
 ('Potential Conflicts Around the Globe for Trump, the Businessman President - The New York Times', 'MANILA  —   On Thanksgiving Day, a